# TP1 : Build the first Deep Learning NN with Keras

Our main goal is to predict the churn rate of a certain bank, that is, the customer who will leave this banking service.

# 1- Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

/home/instadeeprd/miniconda3/envs/learn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 2- Preprocessing data

In [2]:
# Importing the dataset using pandas
dataset = pd.read_csv('data.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 2.1- Exploring data

In [3]:
dataset.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [4]:
dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [5]:
dataset.Gender.value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

## 2.2- Dividing data to features and targets

In [6]:
# Create matrix of features and matrix of target variable.
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [7]:
X[:,1:3]

array([['France', 'Female'],
       ['Spain', 'Female'],
       ['France', 'Female'],
       ...,
       ['France', 'Female'],
       ['Germany', 'Male'],
       ['France', 'Female']], dtype=object)

## 2.3- Encoding data

Country has string labels such as “France, Spain, Germany” while Gender has “Male, Female”. We have to encode this strings into numeric

In [8]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [9]:
X[:,1:3]

array([[0, 0],
       [2, 0],
       [0, 0],
       ...,
       [0, 0],
       [1, 1],
       [0, 0]], dtype=object)

In [10]:
print(labelencoder_X_1.classes_)
print(labelencoder_X_2.classes_)

['France' 'Germany' 'Spain']
['Female' 'Male']


In [11]:
print(labelencoder_X_1.inverse_transform([0,1,2,2,1,0]))
print(labelencoder_X_2.inverse_transform([0,1,1,0]))

['France' 'Germany' 'Spain' 'Spain' 'Germany' 'France']
['Female' 'Male' 'Male' 'Female']


/home/instadeeprd/miniconda3/envs/learn/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/instadeeprd/miniconda3/envs/learn/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## 2.4- One hot encoding data

Label encoding has introduced new problem in our data. LabelEncoder has replaced France with 0, Germany 1 and Spain 2 but Germany is not higher than France and France is not smaller than Spain so we need to create a dummy variable for Country.

In [12]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [13]:
print(X[0:5,:])

[[0.0000000e+00 0.0000000e+00 6.1900000e+02 0.0000000e+00 4.2000000e+01
  2.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 0.0000000e+00 4.1000000e+01
  1.0000000e+00 8.3807860e+04 1.0000000e+00 0.0000000e+00 1.0000000e+00
  1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 0.0000000e+00 4.2000000e+01
  8.0000000e+00 1.5966080e+05 3.0000000e+00 1.0000000e+00 0.0000000e+00
  1.1393157e+05]
 [0.0000000e+00 0.0000000e+00 6.9900000e+02 0.0000000e+00 3.9000000e+01
  1.0000000e+00 0.0000000e+00 2.0000000e+00 0.0000000e+00 0.0000000e+00
  9.3826630e+04]
 [0.0000000e+00 1.0000000e+00 8.5000000e+02 0.0000000e+00 4.3000000e+01
  2.0000000e+00 1.2551082e+05 1.0000000e+00 1.0000000e+00 1.0000000e+00
  7.9084100e+04]]


In [14]:
X[1]

array([0.0000000e+00, 1.0000000e+00, 6.0800000e+02, 0.0000000e+00,
       4.1000000e+01, 1.0000000e+00, 8.3807860e+04, 1.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.1254258e+05])

## 2.5- Dividing data to training and test sets 

In Machine Learning, we always divide our data into training and testing part meaning that we train our model on training data and then we check the accuracy of a model on testing data. Testing your model on testing data will only help you evaluate the efficiency of model.

In [15]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## 2.6- Scaling data

If you carefully observe data, you will find that data is not scaled properly. Some variable has value in thousands while some have value is tens or ones. We don’t want any of our variable to dominate on other so let’s go and scale data.

In [16]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# 3- Creating NN and training it

## 3.1- Building the Neuraul Network

In [17]:
#Initializing Neural Network
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

/home/instadeeprd/miniconda3/envs/learn/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/home/instadeeprd/miniconda3/envs/learn/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
/home/instadeeprd/miniconda3/envs/learn/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


## 3.2- Define all functions for compilation

1- optimizer : the optimization technique for Gradient descent

2- loss : the cost (loss) function

3- metrics : define what metrics are used for evaluation during training

In [18]:
# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4830 - acc: 0.7967
Epoch 2/100
8000/8000 [==============================] - 2s 206us/step - loss: 0.4268 - acc: 0.7967
Epoch 3/100
8000/8000 [==============================] - 3s 339us/step - loss: 0.4211 - acc: 0.7986
Epoch 4/100
8000/8000 [==============================] - 2s 283us/step - loss: 0.4167 - acc: 0.8226
Epoch 5/100
8000/8000 [==============================] - 3s 317us/step - loss: 0.4134 - acc: 0.8275
Epoch 6/100
8000/8000 [==============================] - 2s 284us/step - loss: 0.4119 - acc: 0.8305
Epoch 7/100
8000/8000 [==============================] - 2s 288us/step - loss: 0.4099 - acc: 0.8339
Epoch 8/100
8000/8000 [==============================] - 2s 228us/step - loss: 0.4081 - acc: 0.8334
Epoch 9/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.4071 - acc: 0.8332
Epoch 10/100
8000/8000 [==============================] - 2s 266us/step - loss: 0.4056 - acc: 0.8350
E

## 3.3- Predicting the Test set results

In [20]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [ True]
 [False]
 ...
 [False]
 [False]
 [False]]


## 3.4- Creating the Confusion Matrix

![title](Img/confusion_matrix.png)

In [21]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1552   37]
 [ 294  117]]


In [22]:
# Accuracy of our model
accuracy = (cm[0,0]+cm[1,1]) * 100/ cm.sum()
print("The accuracy of our model is %2.1f %%" % accuracy)

The accuracy of our model is 83.5 %
